In [32]:
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
import pandas as pd
from pandas import DataFrame, Series
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
sns.set(style='ticks', palette='Set2')
%matplotlib inline

In [33]:
# Generate data
np.random.seed(7)

n_samples, n_features = 100, 200
X = np.random.randn(n_samples, n_features)

k = 5
# beta generated with k nonzeros
#coef = 10 * np.random.randn(n_features)
coef = 10 * np.ones(n_features)
inds = np.arange(n_features)
np.random.shuffle(inds)
coef[inds[k:]] = 0  # sparsify coef
y = np.dot(X, coef)

# add noise
y += 0.01 * np.random.normal((n_samples,))

# Split data in train set and test set
n_samples = X.shape[0]
X_train, y_train = X[:25], y[:25]
X_test, y_test = X[25:], y[25:]

In [34]:
# Problem 2

In [76]:
# 2.1
# from sklearn.feature_selection import RFECV
# from sklearn.linear_model import Lasso

# selected_features = []

# features_r2 = []

# for i in range(0,201):
#     clf = linear_model.LinearRegression()
#     clf.fit(X_train[:,i].reshape(-1, 1), y_train)
#     features_r2.append(r2_score(y_test, clf.predict(X_test)) )

# print(features_r2)


# alpha = 3
# lasso = Lasso(alpha=alpha)

# y_pred_lasso = lasso.fit(X_train, y_train).predict(X_test)
# r2_score_lasso = r2_score(y_test, y_pred_lasso)
# print("R^2 on test data : %f" % r2_score_lasso)


# rfecv = RFECV(estimator=lasso, step=1, cv=5, scoring='accuracy')
# rfecv = rfecv.fit(X_train, y_train)
# print('Optimal number of features :', rfecv.n_features_)
# print('Best features :', x_train.columns[rfecv.support_])



ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 1 is different from 200)

In [88]:
# 2.3
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.linear_model import Lasso

alphas = np.logspace(-4, -0.5, 30)
lasso = Lasso(random_state=0, max_iter=10000)

tuned_parameters = [{'alpha': alphas}]
n_folds = 5

clf = GridSearchCV(lasso, tuned_parameters, cv=n_folds, scoring='r2')

clf.fit(X, y)

print("Best Alpha:", clf.best_params_)
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']

for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
        
print("The best alpha is: 0.0006995642156712634")

Best Alpha: {'alpha': 0.0006995642156712634}
0.181 (+/-0.823) for {'alpha': 0.0001}
0.181 (+/-0.822) for {'alpha': 0.00013203517797162948}
0.182 (+/-0.822) for {'alpha': 0.00017433288221999874}
0.189 (+/-0.829) for {'alpha': 0.00023018073130224678}
0.596 (+/-0.472) for {'alpha': 0.0003039195382313198}
1.000 (+/-0.000) for {'alpha': 0.0004012807031942776}
1.000 (+/-0.000) for {'alpha': 0.0005298316906283707}
1.000 (+/-0.000) for {'alpha': 0.0006995642156712634}
1.000 (+/-0.000) for {'alpha': 0.0009236708571873865}
1.000 (+/-0.000) for {'alpha': 0.0012195704601594415}
1.000 (+/-0.000) for {'alpha': 0.0016102620275609393}
1.000 (+/-0.000) for {'alpha': 0.0021261123338996556}
1.000 (+/-0.000) for {'alpha': 0.0028072162039411755}
1.000 (+/-0.000) for {'alpha': 0.0037065129109221566}
1.000 (+/-0.000) for {'alpha': 0.004893900918477494}
1.000 (+/-0.000) for {'alpha': 0.006461670787466976}
1.000 (+/-0.000) for {'alpha': 0.008531678524172814}
1.000 (+/-0.000) for {'alpha': 0.011264816923358867}

In [89]:
# 2.4

for n_fold in range(1,10):
    print("Number of folds: ", n_fold)
    clf = GridSearchCV(lasso, tuned_parameters, cv=n_folds, scoring='r2')
    clf.fit(X, y)
    print("Best Alpha:", clf.best_params_)
    
print("As the number of folds increases, the hyper parameter stays the same.")

Number of folds:  1
Best Alpha: {'alpha': 0.0006995642156712634}
Number of folds:  2
Best Alpha: {'alpha': 0.0006995642156712634}
Number of folds:  3
Best Alpha: {'alpha': 0.0006995642156712634}
Number of folds:  4
Best Alpha: {'alpha': 0.0006995642156712634}
Number of folds:  5
Best Alpha: {'alpha': 0.0006995642156712634}
Number of folds:  6
Best Alpha: {'alpha': 0.0006995642156712634}
Number of folds:  7
Best Alpha: {'alpha': 0.0006995642156712634}
Number of folds:  8
Best Alpha: {'alpha': 0.0006995642156712634}
Number of folds:  9
Best Alpha: {'alpha': 0.0006995642156712634}
As the number of folds increases, the hyper parameter stays the same.


# 2.5

LassoCV will choose a set of alphas and perform comparison searches over then, using cross-validation to generate lasso models and internally score the results. The previous step used GridsearchCV + Lasso model, which is essentially performing the same functions as LassoCV. The results from LassoCV and GridsearchCV + Lasso should agree with each other if the parameters were configured the same way.